### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [17]:
%%time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, \
                        accuracy_score, classification_report, roc_auc_score, roc_curve

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 17.9 µs


## Шаг 1. Импортируем все в Pandas


In [18]:
%%time
sample_submission = pd.read_csv('sampleSubmission.csv',',')
train_df = pd.read_csv('train.csv','\t')
test_df = pd.read_csv('test.csv','\t')
other_df = pd.read_csv('other.csv','\t')

CPU times: user 18.5 s, sys: 2.21 s, total: 20.7 s
Wall time: 20.9 s


In [19]:
train_df['concat'] = train_df['name']+' '+ train_df['description']
train_df.head()

,id,name,description,target,concat
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,Заведующий отделом/секцией в магазин YORK (Уру...
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Наладчик станков и манипуляторов с ПУ Обязанно...
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Разработчик С++ (Криптограф) <strong>Требовани...
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Мерчендайзер/продавец-консультант <p><strong>К...


In [5]:
train_df.columns.values

array(['id', 'name', 'description', 'target', 'concat'], dtype=object)

In [20]:
print(train_df['concat'][0])

Заведующий отделом/секцией в магазин YORK (Уручье) <p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары).</strong></p> <p><strong>Обязанности: </strong></p> <p> </p> <ul> <li>организация эффективного и культурного обслуживания покупателей</li> <li>организация приемки-сдачи товаров на склад</li> <li>консультация покупателей по вопросам, касающимся оказываемых услуг</li> <li>контроль своевременной подачи товаров в торговую секцию</li> <li>проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах</li> <li>организация бесперебойной работы товарной секции и участка склада, закрепленного за ней</li> <li>контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей</li> <li>инвентаризация товаров</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>знание товарной группы</li> <li>знание правил приемки товара</li> <li>опыт работы в розничной торговле в качестве продавца 6 р. / зам

## Шаг 2. Чистим данные и подготавливаем текст

In [21]:
# Подключаем БьютифулСуп
from bs4 import BeautifulSoup             

# Прогоняем отдельное описание через БьютифулСуп   
example1 = BeautifulSoup(train_df['concat'][0], 'html5lib')  

In [22]:
#Подключаем регулярки, чтобы очистить текст от тегов html
import re
# Делаем найти и заменить на ничто для всего что не похоже на регулярку
letters_only = re.sub("[^а-яА-Яa-zA-Z]",           # находим всё что не
                      " ",                   # Меняем на пустоту
                      example1.get_text() )  
print (letters_only)

Заведующий отделом секцией в магазин YORK  Уручье  В НОВЫЙ МАГАЗИН YORK  хозтовары  приглашаем на постоянную работу руководителя секции  хозтовары   Обязанности      организация эффективного и культурного обслуживания покупателей организация приемки сдачи товаров на склад консультация покупателей по вопросам  касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества  сроков годности ТМЦ  наличие маркировок  ценников на товарах организация бесперебойной работы товарной секции и участка склада  закрепленного за ней контроль за сохранностью товаров  торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования   знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца   р    заместителя заведующего секцией отделом ответственность  лидерские качества  опыт управления коллективом      Условия   НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив д

In [23]:
lower_case = letters_only.lower()
words = lower_case.split()  

In [24]:
import nltk

#nltk.download()  # Если NLTK не установлен, то надо раскомментровать этот блок и исполнить

In [25]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
print( stopwords.words('russian') )

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [26]:
# Удаляем стоп-слова русского языка из текста words
words = [w for w in words if not w in stopwords.words('russian')]
print (words)
# Посмоторим что получится, если прогнать стеммером. 
# Но вариант со стеммером будет в следующей версии домашней работы
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer( 'russian' )
words = [snowball_stemmer.stem(w) for w in words if not w in stopwords.words('russian')]
print (words)

['заведующий', 'отделом', 'секцией', 'магазин', 'york', 'уручье', 'новый', 'магазин', 'york', 'хозтовары', 'приглашаем', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары', 'обязанности', 'организация', 'эффективного', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'склад', 'консультация', 'покупателей', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'участка', 'склада', 'закрепленного', 'контроль', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'розничной', 'торговле', 'качестве', 'продавца', 'р', 'заместителя', 'за

In [27]:
def vac_to_words( description, Stemmer ):
    """
    Функция делает преобразование поля description из формата html в формат с отдельными значащими словами
    На вход передается  описание и параметр True/False. Если True, то работает стеммер, иначе нет
    """
    description_text = BeautifulSoup(description, 'html5lib').get_text() #получаем текст      
    letters_only = re.sub('[^а-яА-Яa-zA-Z]', ' ', description_text) #удаляем всё что не буквы
    words = letters_only.lower().split() #получаем слова в нижнем регистре                             
    stops = set(stopwords.words('russian'))  # формируем set стоп слов русского языка                  
    if Stemmer: #удаляем минус фразы и  оставляем либо "стеммы", либо значащие слова
        meaningful_words = [snowball_stemmer.stem(w) for w in words if not w in stops]
    else: 
        meaningful_words = [w for w in words if not w in stops]
    return( ' '.join( meaningful_words ))  # собираем слова в строку с разделителеим пробел 

In [28]:
clean_vac = vac_to_words( train_df['concat'][0] , True)
print(clean_vac)

завед отдел секц магазин york уруч нов магазин york хозтовар приглаша постоя работ руководител секц хозтовар обязан организац эффективн культурн обслуживан покупател организац приемк сдач товар склад консультац покупател вопрос каса оказыва услуг контрол своевремен подач товар торгов секц проверк качеств срок годност тмц налич маркировок ценник товар организац бесперебойн работ товарн секц участк склад закреплен контрол сохран товар торгов оборудован проч материальн ценност инвентаризац товар требован знан товарн групп знан прав приемк товар оп работ розничн торговл качеств продавц р заместител завед секц отдел ответствен лидерск качеств оп управлен коллектив услов нов современ магазин розничн торговл хозтовар хорош коллект достойн зарплат прем соц пакет график работ пн пт плава мест работ ул никифоров уруч


In [29]:
# Посмотрим сколько всего описаний есть в train_df
num_vacs = train_df['concat'].size
num_vacs

200000

In [30]:
%%time
clean_train_vacs = [] #делаем пустой лист для записи всех описаний с отдельными словами

print ('Очищаем и собираем нормализованные описания из train_df...\n')

# Пройдем в цикле по всем описаниям
for i in range( 0, num_vacs ):
    
    # Для каждого описания вызываем функцию нормировки текста описания и добавляем нормированный текст в лист
    
    if( (i+1)%10000 == 0 ): #чтобы не было скучно, каждые 10 000 описаний будем печатать развлекательный текст
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_vacs,(i+1)/num_vacs )) 
        
    clean_train_vacs.append( vac_to_words( train_df['concat'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из train_df...

Описание 10000 из 200000 в обработке...
Описание 20000 из 200000 в обработке...
Описание 30000 из 200000 в обработке...
Описание 40000 из 200000 в обработке...
Описание 50000 из 200000 в обработке...
Описание 60000 из 200000 в обработке...
Описание 70000 из 200000 в обработке...
Описание 80000 из 200000 в обработке...
Описание 90000 из 200000 в обработке...
Описание 100000 из 200000 в обработке...
Описание 110000 из 200000 в обработке...
Описание 120000 из 200000 в обработке...
Описание 130000 из 200000 в обработке...
Описание 140000 из 200000 в обработке...
Описание 150000 из 200000 в обработке...
Описание 160000 из 200000 в обработке...
Описание 170000 из 200000 в обработке...
Описание 180000 из 200000 в обработке...
Описание 190000 из 200000 в обработке...
Описание 200000 из 200000 в обработке...
Я закончила!!!
CPU times: user 27min 47s, sys: 9.85 s, total: 27min 57s
Wall time: 28min


## Создаём свойства через Bag of Words

In [31]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs)

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 15.6 s, sys: 2.59 s, total: 18.2 s
Wall time: 18.7 s


In [32]:
train_data_features.shape

(200000, 5000)

In [33]:
# Посмотрим на получившийся словарь 
vocab = vectorizer.get_feature_names()
print (vocab)

['abap', 'abilit', 'abl', 'about', 'access', 'according', 'account', 'accounts', 'achiev', 'across', 'activ', 'activities', 'ad', 'additiona', 'administration', 'administrator', 'adob', 'advanced', 'advantag', 'advertising', 'adwords', 'after', 'agil', 'ajax', 'algorithms', 'all', 'als', 'always', 'amazon', 'amocrm', 'among', 'an', 'analysis', 'analyst', 'analytica', 'analytics', 'analyz', 'and', 'android', 'angular', 'angularjs', 'annua', 'ansibl', 'ap', 'apach', 'apis', 'app', 'appl', 'application', 'applications', 'approach', 'approaches', 'appropriat', 'apps', 'ar', 'architect', 'architectur', 'are', 'areas', 'arm', 'around', 'art', 'as', 'asp', 'assigned', 'assist', 'assuranc', 'asterisk', 'at', 'atlassia', 'atmospher', 'attention', 'attitud', 'aud', 'autocad', 'automated', 'automation', 'availabilit', 'availabl', 'aws', 'azur', 'bachelor', 'back', 'backbon', 'backend', 'background', 'backup', 'bas', 'based', 'bash', 'basic', 'basis', 'be', 'beaut', 'becom', 'befr', 'being', 'bela

In [34]:
import numpy as np

# Просуммируем счетчики
dist = np.sum(train_data_features, axis=0)

# Посмотрим сколько раз каждое слово встречается в train_df

for tag, count in zip(vocab, dist):
    print (count, tag)

530 abap
3514 abilit
1024 abl
1732 about
513 access
654 according
1314 account
322 accounts
374 achiev
917 across
1109 activ
1420 activities
1475 ad
428 additiona
608 administration
288 administrator
2436 adob
992 advanced
698 advantag
332 advertising
1852 adwords
342 after
2812 agil
1249 ajax
299 algorithms
3032 all
784 als
316 always
528 amazon
306 amocrm
416 among
5150 an
1563 analysis
626 analyst
885 analytica
2460 analytics
507 analyz
74257 and
4865 android
2109 angular
1054 angularjs
376 annua
569 ansibl
3982 ap
1301 apach
339 apis
670 app
1533 appl
1883 application
1827 applications
593 approach
282 approaches
319 appropriat
388 apps
8721 ar
688 architect
1313 architectur
599 are
490 areas
327 arm
764 around
407 art
8355 as
1944 asp
292 assigned
340 assist
365 assuranc
409 asterisk
3394 at
315 atlassia
425 atmospher
331 attention
289 attitud
339 aud
2306 autocad
525 automated
1943 automation
310 availabilit
370 availabl
922 aws
551 azur
470 bachelor
1317 back
307 backbon
1709 ba

437 вентиляцион
1720 вер
412 верн
5123 верс
354 верста
874 верстальщик
3568 верстк
713 вертикальн
2996 верхн
749 вес
1054 весел
8116 вест
1964 ветеринарн
608 вечер
896 вечеринк
1039 вечерн
853 вещ
463 взаимовыгодн
22850 взаимодейств
1209 взаимодействова
1638 взаимоотношен
569 взаиморасчет
526 взам
696 взгляд
398 взросл
1355 взят
12169 вид
3742 виде
401 виден
1264 видеонаблюден
3346 видет
382 виз
1515 визит
1709 визитн
941 визуализац
1576 визуальн
347 виконан
1032 вимог
758 вин
1163 виртуализац
997 виртуальн
507 висок
2512 витрин
450 вищ
467 вк
859 вклад
4622 включ
2884 включа
363 включен
669 вконтакт
1411 вкус
1600 вкусн
1090 владеет
591 владельц
17407 владен
394 владет
382 владеющ
291 владимир
349 владимирск
409 власт
2316 влия
884 влиян
439 вм
7554 вмест
578 вне
361 внедр
18947 внедрен
1733 внедря
2763 внесен
9331 внешн
340 вник
3940 вниман
16357 внимательн
23122 внимательностьуслов
542 внов
1618 внос
4451 внутр
11780 внутрен
768 внутрикорпоративн
717 вовлечен
834 воврем
2133 вод
161

1418 материа
24673 материал
5885 материальн
351 матрас
1549 матриц
4088 машин
408 машинист
1678 машиностроен
867 машиностроительн
8425 мебел
3171 мебельн
1164 мег
567 мегаполис
1258 мегафон
4111 мед
618 медиапланирован
619 медийн
446 медицин
15798 медицинск
457 межвахтов
10576 международн
361 межкомнатн
953 мелк
514 мелоч
20539 мен
74305 менеджер
4435 менеджмент
375 меньш
2051 меня
2850 мер
764 мереж
417 мерл
51797 мероприят
993 мерчандайзер
1403 мерчандайзинг
7724 мерчендайзер
6720 мерчендайзинг
3111 мес
412 мессенджер
35547 мест
596 местн
295 местоположен
1594 месторасположен
58871 месяц
572 месячн
1963 металл
515 металлическ
947 металлоконструкц
294 металлообрабатыва
581 металлообработк
934 металлопрокат
485 металлург
519 металлургическ
8800 метод
2756 методик
1547 методическ
3168 методолог
301 методологическ
9634 метр
2148 метрик
577 метролог
643 метрологическ
465 механизац
2579 механизм
2414 механик
1556 механическ
727 мечт
1584 мечта
1551 ми
621 микроконтроллер
305 микрорайон
427

1998 релиз
1037 релокац
361 релокацион
1102 реляцион
11728 ремонт
1936 ремонтн
748 рентабельн
1012 репутац
2095 республик
1328 рестора
1035 ресторан
5115 ресурс
768 рефакторинг
1110 рецептур
22896 реч
459 речн
1685 реш
5027 реша
40820 решен
373 рза
352 ри
321 риелтор
1721 риск
1447 ритейл
291 ритейлер
330 риэлтор
2421 рк
3886 робот
318 робоч
412 ров
296 род
282 родител
348 родственник
1350 рожден
4615 розниц
46131 розничн
344 розробк
457 рок
1990 рол
476 ролик
467 рос
17871 росс
10945 российск
117698 рост
1433 ростелек
349 ростов
325 ростовск
5130 ротац
331 рощ
302 ртз
524 ру
16161 руб
1307 рубеж
11221 рубл
4604 рук
957 руковод
22963 руководител
15564 руководств
4218 руководя
395 рум
363 румянцев
395 рунет
4291 русск
462 рутин
1427 ручн
95628 рф
434 рыб
363 рыбн
38453 рынк
2798 рынок
867 рыночн
436 рэ
33283 ряд
326 рязан
297 рязанск
407 савеловск
850 сад
494 садов
21070 сайт
11274 салон
15063 сам
1262 самар
389 самарск
455 самодисциплин
500 самообучен
1781 самоорганизац
1109 самооргани

## Шаг 3. Решам задачу через модель Random Forest

In [35]:
#Делаем трейн тест сплит

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

X_train, X_test, y_train, y_test = train_test_split(train_data_features, train_df['target'], test_size=0.33, random_state=42)
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train  (134000, 5000)
X_test  (66000, 5000)
y_train  (134000,)
y_test  (66000,)


In [38]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Пока делаем 100 деревьев, но в следующих вариантах домашки попробую сделать 50
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

# В версии с 5 000 фич обучение заняло 12 минут 41 секунду, старт в 23-25

forest = forest.fit( X_train, y_train )
print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 20min 23s, sys: 20.7 s, total: 20min 43s
Wall time: 2min 24s


In [39]:
predict = forest.predict(X_test)

In [40]:
# прошлый результаты были 0.96 0.96 0.96

print('#'*20,'>>> Random Trees <<<','#'*20,'\n',classification_report(y_test, predict),'\n')

#################### >>> Random Trees <<< #################### 
              precision    recall  f1-score   support

          0       0.97      0.96      0.97     34984
          1       0.96      0.97      0.96     31016

avg / total       0.97      0.97      0.97     66000
 



# Шаг 4. Готовим файл к отправке

In [41]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Делаем random forest на 100 деревьев
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

# В версии с 5 000 фич обучение заняло 22 минут 12 секунд, старт в 23-37
forest = forest.fit( train_data_features, train_df['target'] )

print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 58min 34s, sys: 1min 54s, total: 1h 28s
Wall time: 6min 32s


In [42]:
test_df['concat'] = test_df['name']+' '+ test_df['description']

test_df.shape

(170179, 4)

In [43]:
num_test_vacs = len(test_df['concat'])

In [44]:
%%time
clean_test_vacs = []
print ('Очищаем и собираем нормализованные описания из test_df...\n')
for i in range(0,num_test_vacs):
    if( (i+1) % 10000 == 0 ):
        print ('Описание {} из {} в обработке...'.format( i+1, num_test_vacs)) 
    clean_test_vacs.append( vac_to_words( test_df['concat'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из test_df...

Описание 10000 из 170179 в обработке...
Описание 20000 из 170179 в обработке...
Описание 30000 из 170179 в обработке...
Описание 40000 из 170179 в обработке...
Описание 50000 из 170179 в обработке...
Описание 60000 из 170179 в обработке...
Описание 70000 из 170179 в обработке...
Описание 80000 из 170179 в обработке...
Описание 90000 из 170179 в обработке...
Описание 100000 из 170179 в обработке...
Описание 110000 из 170179 в обработке...
Описание 120000 из 170179 в обработке...
Описание 130000 из 170179 в обработке...
Описание 140000 из 170179 в обработке...
Описание 150000 из 170179 в обработке...
Описание 160000 из 170179 в обработке...
Описание 170000 из 170179 в обработке...
Я закончила!!!
CPU times: user 21min 33s, sys: 6.83 s, total: 21min 40s
Wall time: 21min 44s


In [45]:
%%time
# Фигачим bag of words по тестовым данным

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()

CPU times: user 13.8 s, sys: 2.59 s, total: 16.4 s
Wall time: 16.5 s


In [46]:
%%time
# Через Random Forest делаем предсказание
result_test = forest.predict(test_data_features)

CPU times: user 56.2 s, sys: 3.38 s, total: 59.6 s
Wall time: 9.46 s


In [47]:
# Копируем результат в датафрейм с полями id и target 
# предыдущий реузльтат был таким http://take.ms/cVRK4
output = pd.DataFrame( data={'id':test_df['id'], 'target':result_test} )
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0


In [48]:
output.to_csv( "Random_Forest_5000stem-and-names.csv", sep=',', encoding='utf-8',index=False )